# Benchmark random and pseudo-random solvers on Multi-objective problems

In [3]:
import time
import sys
import datetime
from random import uniform
sys.path.append('..')

# --- Dependencies
import pygmo as pg
import numpy as np
import pandas as pd
import sobol_seq
# import dovpanda

import plotly
from plotly import graph_objs as go
from plotly import tools
import plotly.express as px

from sklearn.model_selection import ParameterGrid
from sklearn.metrics import r2_score

# from src.generator import SamplesGenerator
# from src.composite import PredictTutor, ModelsUnion
from src.ploting import plot_mo
# from src.search import Nsga2
# from src.hypothesis.custom_gp_kernel import KERNEL_MAUNA, KERNEL_SIMPLE, KERNEL_GPML

## Sampling strategies
- Random
- Sobol
- Latin hypercube

In [99]:
def sobol_sample(bounds=([0., 0.], [5., 5.]), n=1):
    """ Sobol sampling

    Args:
        bounds (Tuple):  Tuple with lower and higher bound for each feature in objective space.
        Example: (([0., 0.]), ([2., 4.]))
        n (int, optional): Sample count. Defaults to 1.

    Returns:
        List: 2D list of point(s) from search space
    """
    n_dim = len(bounds[0])
    sb = sobol_seq.i4_sobol_generate(n_dim, n, 1)
    diff = [r-l for l, r in zip(*bounds)]
    left = [l for l, _ in zip(*bounds)]
    return (sb*diff+left).tolist()


def lh_sample(bounds=([0., 0.], [5., 5.]), n=1):
    """ Latin Hypercube sampling

    Args:
        bounds (Tuple):  Tuple with lower and higher bound for each feature in objective space.
        Example: (([0., 0.]), ([2., 4.]))
        n (int, optional): Sample count. Defaults to 1.

    Returns:
        List: 2D list of point(s) from search space
    """
    n_dim = len(bounds[0])
    h_cube = np.random.uniform(size=[n, n_dim])
    for i in range(0, n_dim):
        h_cube[:, i] = (np.argsort(h_cube[:, i])+0.5)/n
    diff = [r-l for l, r in zip(*bounds)]
    left = [l for l, _ in zip(*bounds)]
    return (h_cube*diff+left).tolist()
    
def random_sample(bounds=([0., 0.], [5., 5.]), n=1):
    """ Random sampling

    Args:
        bounds (Tuple):  Tuple with lower and higher bound for each feature in objective space.
        Example: (([0., 0.]), ([2., 4.]))
        n (int, optional): Sample count. Defaults to 1.

    Returns:
        List: 2D list of point(s) from search space
    """
    v_uniform = np.vectorize(uniform)
    return [v_uniform(*bounds).tolist() for _ in range(n)]

In [35]:
# px.scatter(pd.DataFrame(random_sample(n=20), columns=['param 1', 'param 2']), x='param 1', y='param 2', template="presentation", marginal_y="rug", marginal_x="rug", title="Random sampling")

In [36]:
# px.scatter(pd.DataFrame(sobol_sample(n=20), columns=['param 1', 'param 2']), x='param 1', y='param 2', template="presentation", marginal_y="rug", marginal_x="rug", title="Sobol sampling")

In [37]:
# px.scatter(pd.DataFrame(lh_sample(n=20), columns=['param 1', 'param 2']), x='param 1', y='param 2', template="presentation", marginal_y="rug", marginal_x="rug", title="Latin Hypercube sampling")

## Benchmark function

In [156]:
def make_nd_pop(pro, x, y):
    nd_front = pg.fast_non_dominated_sorting(y)[0][0]
    nd_x = np.array(x)[nd_front]
    nd_y = np.array(y)[nd_front]
    t_pop = pg.population(pro)
    for i, p_vector in enumerate(nd_x):
        t_pop.push_back(x=p_vector, f=nd_y[i])
    return t_pop

def experiment(problem_name: str,
               prob_id: int,
               prob_dim: int,
               obj: int,
               random_points: int,
               sampling_plan: str,
               seed=None):
    
    np.random.seed(seed)

    result = {
        "problem_name": problem_name,
        "seed": seed,
        "problem_id": prob_id,
        "objectives": obj,
        "feature_dim": prob_dim,
        "random_points": random_points,
        "sampling_plan": sampling_plan,
        "ndf_x": '',
        "ndf_f": '',
        "fevals": '',
        "evolve_time": '',
        "date": '',
        "p_distance": '',
        "hypervolume": '',
        "ndf_space": '',
        "ndf_size": '',
        'error': '',
        'final': False
    }

    # ----------------------                                                            Initialize problem
    try:
        if problem_name is 'wfg':
            udp = pg.wfg(prob_id=prob_id, dim_dvs=prob_dim,
                         dim_obj=obj, dim_k=obj-1)
        elif problem_name is 'zdt':
            udp = pg.zdt(prob_id=prob_id, param=prob_dim)
        elif problem_name is 'dtlz':
            udp = pg.dtlz(prob_id=prob_id, dim=prob_dim, fdim=obj)
        prob = pg.problem(udp)
    except Exception as err:
        result['error'] = "Init problem: {}".format(err)
        return result

    # ----------------------                                                            Sampling plan initialization
    try:
        points = None
        if sampling_plan is 'random':
            points = random_sample(bounds=prob.get_bounds(), n=random_points)
        elif sampling_plan is 'sobol':
            points = sobol_sample(bounds=prob.get_bounds(), n=random_points)
        elif sampling_plan is 'latin':
            points = lh_sample(bounds=prob.get_bounds(), n=random_points)
        else:
            raise ValueError(f"{sampling_plan}. Parameter error! Acceptable values: 'random','sobol','latin'")
    except Exception as err:
        result['error'] = "Sampling plan initialization: {}".format(err)
        return result

    
    # ----------------------                                                            Solving
    evolve_start = time.time()
    try:
        points_f = pg.bfe()(prob, np.array(points).flatten()).reshape(-1, obj).tolist()
        
        result["fevals"] = prob.get_fevals()
        #This returns the first (i.e., best) non-dominated individual from population:
        nd_pop = make_nd_pop(prob, points, points_f)
        
        score = udp.p_distance(nd_pop) if hasattr(udp, 'p_distance') else None
        
        result["p_distance"] = score or None
        result["ndf_x"] = nd_pop.get_x().tolist()
        result["ndf_f"] = nd_pop.get_f().tolist()
        result["ndf_size"] = len(nd_pop.get_f())
    except Exception as err:
        result['error'] = "Evolve: {}".format(err)
        return result

    # ----------------------                                                            Hypervolume
    try:
        hypervolume = pg.hypervolume(-nd_pop.get_f()
                                    ).compute([0]*nd_pop.problem.get_nobj())
        result['hypervolume'] = hypervolume or None
    except Exception as err:
        result['error'] = "Hypervolume: {}".format(err)
        return result
    
    # ----------------------                                                            Spacing metric
    # The spacing metric aims at assessing the spread (distribution)
    # of vectors throughout the set of nondominated solutions. 
    try:
        dist = pg.crowding_distance(points = nd_pop.get_f())
        not_inf_dist = dist[np.isfinite(dist)]
        mean_dist = np.mean(not_inf_dist)
        space_m = (sum([(mean_dist - d)**2 for d in not_inf_dist])/(len(not_inf_dist)-1))**(1/2)
        result["ndf_space"] = space_m
    except Exception as err:
        result['error'] = "Spacing metric: {}".format(err)
        return result

    # ----------------------                                                            Write results
    
    try:
        t_end = time.time()

        result["problem_name"] = nd_pop.problem.get_name()
        result["objectives"] = nd_pop.problem.get_nobj()
        result["feature_dim"] = nd_pop.problem.get_nx()
        result["evolve_time"] = t_end - evolve_start
        result["date"] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
        result["final"] = True
        result["sampling_plan"] = sampling_plan
        

    except Exception as err:
        result['error'] = "Write results: {}".format(err)

    return result

## Parameter grid

In [157]:
param_grid = {
            'problem_name': ['wfg'],
            'prob_id': [4],
            'prob_dim': [2],
            'obj': [2],
            'random_points': [300,10000],
            'sampling_plan': ['latin'],
            'seed': [774]
        }

res = []
i = 0
for p in ParameterGrid(param_grid):
    i = i+1
    print("\n Evaluation.: {} \n i: {}".format(p, i))
    res.append(experiment(**p))

pd.DataFrame(res)


 Evaluation.: {'obj': 2, 'prob_dim': 2, 'prob_id': 4, 'problem_name': 'wfg', 'random_points': 300, 'sampling_plan': 'latin', 'seed': 774} 
 i: 1

 Evaluation.: {'obj': 2, 'prob_dim': 2, 'prob_id': 4, 'problem_name': 'wfg', 'random_points': 10000, 'sampling_plan': 'latin', 'seed': 774} 
 i: 2


,problem_name,seed,problem_id,objectives,feature_dim,random_points,sampling_plan,ndf_x,ndf_f,fevals,evolve_time,date,p_distance,hypervolume,ndf_space,ndf_size,error,final
0,WFG4,774,4,2,2,300,latin,"[[1.99, 1.82], [1.5566666666666666, 1.58], [0....","[[2.0112727798012315, 0.5819218461811542], [1....",300,0.025075,2020-02-17 17:41,None,6.559020,0.082757,32,,True
1,WFG4,774,4,2,2,10000,latin,"[[0.7133, 1.5726], [1.4763, 1.401], [0.1547, 1...","[[0.36426820467833093, 3.9394922364029648], [1...",10000,2.585008,2020-02-17 17:41,None,6.295351,0.017210,138,,True
